Zixuan Chen 61665307

# Set-up

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from time import perf_counter as clock
from beautifultable import BeautifulTable
np.random.seed(0)

This Function object is borrowed and modified from my own hw2 code. Now it does not use sympy.Symbols so it is more memory efficient  and does the same things. I use this object to save effort in calculating gradients and printing results later on in this program.\
Its differentiate method is implemented using sympy. It also remembers the position of variables in a vector so it is a lot easier for me to calculate the gradient.

In [2]:
class Function:
    def __init__(self, func_str, symbols, name = '', parameter_pos = None):
        '''
            (1) symbols are passed in as a string and seperated by ','
            (2) parameter_pos remembers which variable the function should take. 
                Default is None because the original function takes all variables.
                It is only useful for its partial derivatives functions.
        '''
        self.func = func_str
        self.symbols = [s.strip() for s in symbols.split(',')]
        self.s = len(self.symbols)
        self.name = name
        exec(f'self._f = lambda {symbols}: {func_str}')
        
        self.pos = parameter_pos
        if not self.pos:
            self.pos = list(range(self.s))       
    
    def __repr__(self):
        return self.func
    
    def __call__(self, variables):
        ''' to call the function, we only take what need in the variable vector '''
        if len(variables.shape) == 1: # (n,)
            s = [variables[i] for i in self.pos]
            return self._f(*s)
        elif variables.shape[0] == 1: # (1, n)
            s = [variables[0][i] for i in self.pos]
            return self._f(*s)
        raise AssertionError(f'Dimension dismatch {variables.shape}')
    
    def diff(self, symbol):
        ''' differentiate using sympy, remember all the varibale positions and return a new Function object '''
        df = str(sp.diff(self.func, symbol))  #str(sp.Function(str(sp.diff(self.func, symbol))))
        sym, pos = [], []
        for i, s in enumerate(self.symbols):
            if s in df:
                sym.append(s)
                pos.append(i)
        return Function(df, ','.join(sym), parameter_pos = pos)

These helper functions are also borrowed and modified from hw2. I use them to reduce potential duplicate typing later on in the program.\
New: "assert_shape", a sanity checker to ensure the dimensionality of vectors.

In [3]:
def gradient(f:'Function') -> ['Function']:
    ''' return a gradient vector'''
    return np.array([f.diff(s) for s in f.symbols])# (n,)

def compute_gradient(g: ['Function'], x: 'np.array') -> ['value']:
    ''' compute the gradient value at given point'''
    return np.array([d(x) for d in g], ndmin = 2)  # (1, n)

def gradient_magnitude(g, x):
    ''' return the length of the gradient vector at point/vector x'''
    if len(g.shape) == 1: # (n,)
        return np.sqrt(sum([i(x)**2 for i in g]))
    elif g.shape[0] == 1: # (1, n)
        return np.sqrt(sum([i(x)**2 for i in g[0]]))
    raise AssertionError(f'Dimension dismatch {g.shape}')
        
def vector_length(v):
    ''' return the length of a vector'''
    if len(v.shape) == 1: # (n,)
        return np.sqrt(sum([i**2 for i in v]))
    elif v.shape[0] == 1: # (1, n)
        return np.sqrt(sum([i**2 for i in v[0]]))
    raise AssertionError(f'Dimension dismatch {v.shape}')
    
# sanity checker
def assert_shape(np_obj, shape):
    assert np_obj.shape == shape, str(np_obj.shape)

This code of line search methods comes from hw1 and they are all provided in the Kochenderfer and Wheeler book:\
"initial_min_bracket" is from Algorithm 3.1, "bracket_minimum" (pg.36).\
"golden_section" is from Algorithm 3.3, "golden_section_search" (pg.41).\
"line_search" is from Algorithm 4.1, "line_search" (pg.54). It use the two methods above to compute a step size.\
I will use this method to compute the step size in Quasi-Newton's method.

In [4]:
def initial_min_bracket(f, start = 0, step = 1e-2, expand_factor = 2.0):
    '''Algorithm 3.1'''
    a, fa = start, f(start)
    b, fb = start + step, f(start + step)
    if fb > fa:
        a, b = b, a
        fa, fb = fb, fa
        step = -step
    while True:
        c, fc = b + step, f(b + step)
        if fc > fb:
            return (a, c) if a <= c else (c, a)
        a, fa, b, fb = b, fb, c, fc
        step *= expand_factor
        
def golden_section(func, a, b, tol = 1e-6):
    '''Algorithm 3.3'''
    phi = (1 + np.sqrt(5))/2 - 1
    d = phi * (b - a) + a  
    fd = func(d)
    
    while abs(b - a) > tol:
        c = b - phi * (b - a) 
        fc = func(c)
        if fc < fd:
            b, d, fd = d, c, fc
        else:
            a, b = b, c
    return (a + b) / 2

def line_search(f, x, direction):
    '''Algorithm 4.1'''
    objective = lambda alpha : f(x + alpha * direction)
    a, b = initial_min_bracket(objective)
    return golden_section(objective, a, b)

# Problem 1 Quasi-Newton method

Note: both DFP and BFGS are slightly modified because my vectors' dimensionality is different from what the book defined; mine are 1\*n, but the Kochenderfer and Wheeler book use n\*1. To resolve this difference, all I need to do is to transpose every vector except Q in the original formula.

Algorithm 6.3. The Davidon-Fletcher-Powell descent method. Kochenderfer and Wheeler, pg.93

In [5]:
def DFP_method(Q: 'np.array', delta: 'np.array', gama: 'np.array', dim = 10):
    # do matrix operations
    delta, gama = np.mat(delta), np.mat(gama)
    
    a = Q* gama.T * gama * Q ; assert_shape(a, (dim,dim))
    b = gama * Q * gama.T ; assert_shape(b, (1,1))
    c = delta.T * (delta) ; assert_shape(c, (dim,dim))
    d = delta * (gama.T) ; assert_shape(d, (1,1))
    if np.asscalar(b) != 0 and np.asscalar(d) != 0: # avoid 0 division
        Q -= a /b + c / d
    return np.asarray(Q) # back to array

Algorithm 6.4. The Broyden-Fletcher-Goldfarb-Shanno descent method. Kochenderfer and Wheeler, pg.93

In [6]:
def BFGS_method(Q: 'np.array', delta: 'np.array', gama: 'np.array', dim = 10):
    # do matrix operations
    delta, gama = np.mat(delta), np.mat(gama)
    
    d_g = delta * gama.T ; assert_shape(d_g, (1,1))
    if np.asscalar(d_g) == 0: # avoid 0 division
        return Q
    d_g_Q = delta.T *(gama) * Q; assert_shape(d_g_Q, (dim,dim))
    Q_g_d = Q * gama.T *(delta); assert_shape(Q_g_d, (dim,dim))
    g_Q_g = gama * Q * (gama.T); assert_shape(g_Q_g, (1,1))
    d_d = delta.T * (delta); assert_shape(d_d, (dim,dim))
    
    a = (d_g_Q + Q_g_d) / d_g
    b = d_d/d_g + np.asscalar(g_Q_g) * d_d / d_g**2
    return np.asarray(Q - a + b) # back to array

The process of quasi_newton optimization:\
(1) initialize\
(2) if the gradient magnitude is smaller than the tolerance, go to (10), otherwise continue:\
(3) update iteration counter\
(4) update direction\
(5) find step size using line search\
(6) update x\
(7) if the convergence is very small, or the iteration count is too large, it likely marks a failure. Report failure and break to (10), otherwise continue\
(8) update gradient\
(9) update approximated inverse Hessian, Q, using change of x and change of gradient. Then go back to (2)\
(10) Iteration ends. Return a result consisting of ||delta||, |f(x)|, iteration counter and running time

In [7]:
def quasi_newton(f : 'Function', initial_point : 'np.array', fprime : ['Function'] , method = BFGS_method, tol = 1e-4, dim = 10):
    # initializations
    start_time = clock()
    x = initial_point.copy()
    Q = np.identity(x.shape[1])  # inverse Hessian initially be a identity matrix
    iteration, delta = 0, np.zeros((1,dim))
    
    # main loop
    g = compute_gradient(fprime, x)
    while vector_length(g) > tol:  # stopping criterion: gradient magnitude
        iteration += 1
        # update x
        direction = -g.dot(Q)
        assert_shape(direction, (1, dim))
        step = line_search(f, x, direction)
        delta = step * direction; assert_shape(delta, (1, dim))
        x += delta
        if vector_length(delta) < 1e-14 or iteration > 3000:  # break early when likely encountering a failure
            print('Failure. last step size:', step, ' iteration:', iteration)
            break
            
        # update Q, esitimated inverse Hessian
        g_prev = g
        g = compute_gradient(fprime, x)
        gama = g - g_prev; assert_shape(gama, (1, dim))
        Q = method(Q, delta, gama, dim); assert_shape(Q, (dim, dim))
    return vector_length(delta), abs(f(x)), iteration, clock() - start_time

# Test Quasi-Newton method

Here I define two test functions.\
The first one is a 10-dim version of the quadratic function (x-1)^2\
The second one is a 10-dim Rosenbrock function with a = 1 and b = 5

In [8]:
# 10-dim (x-1)^2
f_str = ' + '.join([f'(x{i} - 1)**2' for i in range(10)])
test_func1 = Function(f_str, ','.join([f'x{i}' for i in range(10)]), name = 'Quadratic')

# 10-dim Rosenbrock 
f_str = ' + '.join([f'(1 - x{i})**2 + 5*(x{i+1} - x{i}**2)**2' for i in range(9)])
test_func2 = Function(f_str, ','.join([f'x{i}' for i in range(10)]), name = 'Rosenbrock')

for i, tf in enumerate([test_func1, test_func2], 1):
    print(f'Test Function {i}: {tf.name}\n{tf}\nvariables: {tf.symbols}\n')

Test Function 1: Quadratic
(x0 - 1)**2 + (x1 - 1)**2 + (x2 - 1)**2 + (x3 - 1)**2 + (x4 - 1)**2 + (x5 - 1)**2 + (x6 - 1)**2 + (x7 - 1)**2 + (x8 - 1)**2 + (x9 - 1)**2
variables: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']

Test Function 2: Rosenbrock
(1 - x0)**2 + 5*(x1 - x0**2)**2 + (1 - x1)**2 + 5*(x2 - x1**2)**2 + (1 - x2)**2 + 5*(x3 - x2**2)**2 + (1 - x3)**2 + 5*(x4 - x3**2)**2 + (1 - x4)**2 + 5*(x5 - x4**2)**2 + (1 - x5)**2 + 5*(x6 - x5**2)**2 + (1 - x6)**2 + 5*(x7 - x6**2)**2 + (1 - x7)**2 + 5*(x8 - x7**2)**2 + (1 - x8)**2 + 5*(x9 - x8**2)**2
variables: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']



The test function defined below will optimize a given function over N starting points, using a given method, and report the results using a BeautifulTable.\
The table include a sample mean and an unbiased estimate of population std for each of Convergence Measure, absolute error from the global solution, iteration number and running time.\
In this problem, I set N = 50 and each point is a 10-dim vector ranging from -100 to 100.

In [9]:
def test(func, init_pt, func_gradient, method, sample_size):
    result = np.array([quasi_newton(func, init_pt[i], func_gradient, method) for i in range(sample_size)])
    
    table = BeautifulTable()
    table.set_style(BeautifulTable.STYLE_COMPACT)
    table.column_headers = [func.name]
    
    report = BeautifulTable()
    table.append_row([report])
    report.column_headers = ["Convergence Measure\n||x` - x||", "|error|\n|f(x)|", "Iteration", "time"]
    
    row = []
    for i in range(result.shape[1]):
        m = np.mean(result[:,i])  # sample mean
        s = np.sqrt(sum([(k - m)**2 for k in result[:,i]]) / (sample_size - 1)) # unbiased estimate of population std
        row.append('mean:{:.3e}\nstd:{:.3e}'.format(m,s))
    report.append_row(row)
    return table

In [10]:
sample_size = 50
N = np.random.uniform(-100, 100, (sample_size, 1, 10))

for method in [DFP_method, BFGS_method]:
    for func in [test_func1, test_func2]:
        print('Method used: ', method.__name__)
        print(test(func, N, gradient(func), method, sample_size))
        print() # empty line

Method used:  DFP_method
                                 Quadratic                                  
----------------------------------------------------------------------------
 +---------------------+----------------+----------------+----------------+ 
 | Convergence Measure |    |error|     |   Iteration    |      time      | 
 |     ||x` - x||      |     |f(x)|     |                |                | 
 +---------------------+----------------+----------------+----------------+ 
 |   mean:1.819e+02    | mean:1.415e-10 | mean:1.000e+00 | mean:6.931e-04 | 
 |    std:3.043e+01    | std:4.403e-11  | std:0.000e+00  | std:3.131e-04  | 
 +---------------------+----------------+----------------+----------------+ 

Method used:  DFP_method
                                 Rosenbrock                                 
----------------------------------------------------------------------------
 +---------------------+----------------+----------------+----------------+ 
 | Convergence Measure | 

Observation:\
Because the direction is not normalized, the Quasi-Newton works very well on the quadratic function. It takes only one step to reach a global minimum.\
It has a relatively large error on the Rosenbrock's function because in high dimensions (d > 3), depending on the starting point, it might converge to a local minimum near (-1, 1, ... 1) (Source: Piazza@24 and Wikipedia).\
Overall, BFGS has better performance than DFP; it takes fewer iterations to converge and cost less running time. But the solutions they produce are similar.